# Pymaceuticals Inc.
---

### Analysis

- Add your analysis here.
 

## Background
<h4>
You've just joined Pymaceuticals, Inc., a new pharmaceutical company that specializes in anti-cancer medications.  
Recently, it began screening for potential treatments for squamous cell carcinoma (SCC), a commonly occurring form of skin cancer.  

As a senior data analyst at the company, you've been given access to the complete data from their most recent  
animal study. In this study, 249 mice who were identified with SCC tumors received treatment with a range of drug regimens.  
Over the course of 45 days, tumor development was observed and measured. The purpose of this study was to compare the  
performance of Pymaceuticals’ drug of interest, Capomulin, against the other treatment regimens.  

The executive team has tasked you with generating all of the tables and figures needed for the technical report of the  
clinical study. They have also asked you for a top-level summary of the study results.  </h4>

## Instructions

### This assignment is broken down into the following tasks:  
#### Prepare the data.  
#### |-+-| Generate summary statistics.  
#### |-+-| Create bar charts and pie charts.  
#### |-+-| Calculate quartiles, find outliers, and create a box plot.  
#### |-+-| Create a line plot and a scatter plot.  
#### |-+-| Calculate correlation and regression.  
#### |-+-| Submit your final analysis.  

In [51]:
# LIBRARY AND SETUP:

# Import dependencies
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.stats as st

# Path assignments
mouseData_path = Path('data/Mouse_metadata.csv')
studyData_path = Path('data/Study_results.csv')

# Read-in CSV's and convert to DataFrame objects
mouseData_df = pd.read_csv(mouseData_path)
studyData_df = pd.read_csv(studyData_path)

# Merge DataFrames and preview
mergeData_df = pd.merge(mouseData_df, studyData_df, how = 'outer', on = 'Mouse ID')
mergeData_df.head(10)

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,a203,Infubinol,Female,20,23,0,45.000000,0
1,a203,Infubinol,Female,20,23,5,48.508468,0
2,a203,Infubinol,Female,20,23,10,51.852437,1
3,a203,Infubinol,Female,20,23,15,52.777870,1
4,a203,Infubinol,Female,20,23,20,55.173336,1
5,a203,Infubinol,Female,20,23,25,56.793208,1
6,a203,Infubinol,Female,20,23,30,59.523197,1
7,a203,Infubinol,Female,20,23,35,61.931650,2
8,a203,Infubinol,Female,20,23,40,63.593490,2
9,a203,Infubinol,Female,20,23,45,67.973419,2


In [ ]:
# DATAFRAME CHECKING AND CLEANUP:

# Check # of mice in df (unique Mouse ID's)
mouse_count = len(mergeData_df['Mouse ID'].unique())

# Data entries will be uniquely identified by Mouse ID and Timepoint
# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint.
# grouped_mouseID = mergeData_df.groupby('Mouse ID')['Timepoint']
# print(mergeData_df.at[3, 'Mouse ID'])
timepoints = []
duplicates = {}
for mouse in mouse_count:
    for row in mergeData_df:


# Optional: Get all the data for the duplicate mouse ID.


a203
